<a href="https://colab.research.google.com/github/preetamjumech/LLM/blob/main/TokenizersFromScratch_WordpieceTokenizer_02_11_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from datasets import load_dataset

# Load AG News dataset
ag_news_dataset = load_dataset("ag_news", split = "train[:150]")

ag_news_dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/8.07k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'label'],
    num_rows: 150
})

In [3]:
text_samples = [example['text'] for example in ag_news_dataset]

text_samples[:5]

["Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.",
 'Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\\which has a reputation for making well-timed and occasionally\\controversial plays in the defense industry, has quietly placed\\its bets on another part of the market.',
 "Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\\about the economy and the outlook for earnings are expected to\\hang over the stock market next week during the depth of the\\summer doldrums.",
 'Iraq Halts Oil Exports from Main Southern Pipeline (Reuters) Reuters - Authorities have halted oil export\\flows from the main pipeline in southern Iraq after\\intelligence showed a rebel militia could strike\\infrastructure, an oil official said on Saturday.',
 'Oil prices soar to all-time record, posing new menace to US economy (A

In [4]:
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)

tokenizer = Tokenizer(models.WordPiece(unk_token = "[UNK]"))

tokenizer

In [5]:
tokenizer.normalizer = normalizers.BertNormalizer(lowercase = False)

tokenizer.normalizer.normalize_str("The pièce de résistance was the chef's Special Dessert.")

"The pièce de résistance was the chef's Special Dessert."

In [6]:
tokenizer.normalizer = normalizers.BertNormalizer(lowercase = True)

tokenizer.normalizer.normalize_str("The pièce de résistance was the chef's Special Dessert.")

"the piece de resistance was the chef's special dessert."

In [7]:
tokenizer.normalizer = normalizers.Sequence(
    [normalizers.NFD(), normalizers.Lowercase(), normalizers.StripAccents()]
)

tokenizer.normalizer.normalize_str("  The pièce de résistance was the chef's Special Dessert.   ")

"  the piece de resistance was the chef's special dessert.   "

In [8]:
tokenizer.normalizer = normalizers.Sequence(
    [normalizers.Strip(), normalizers.NFD(), normalizers.Lowercase(), normalizers.StripAccents()]
)

tokenizer.normalizer.normalize_str("   The pièce de résistance was the chef's Special Dessert.   ")

"the piece de resistance was the chef's special dessert."

In [9]:
tokenizer.pre_tokenizer = pre_tokenizers.BertPreTokenizer()

tokenizer.pre_tokenizer.pre_tokenize_str("We're checking pre-tokenization step.")

[('We', (0, 2)),
 ("'", (2, 3)),
 ('re', (3, 5)),
 ('checking', (6, 14)),
 ('pre', (15, 18)),
 ('-', (18, 19)),
 ('tokenization', (19, 31)),
 ('step', (32, 36)),
 ('.', (36, 37))]

In [10]:
tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()

tokenizer.pre_tokenizer.pre_tokenize_str("We're checking the pre-tokenization step.")

[('We', (0, 2)),
 ("'", (2, 3)),
 ('re', (3, 5)),
 ('checking', (6, 14)),
 ('the', (15, 18)),
 ('pre', (19, 22)),
 ('-', (22, 23)),
 ('tokenization', (23, 35)),
 ('step', (36, 40)),
 ('.', (40, 41))]

In [11]:
tokenizer.pre_tokenizer.pre_tokenize_str("We're checking the pretokenization step.")

[('We', (0, 2)),
 ("'", (2, 3)),
 ('re', (3, 5)),
 ('checking', (6, 14)),
 ('the', (15, 18)),
 ('pretokenization', (19, 34)),
 ('step', (35, 39)),
 ('.', (39, 40))]

In [12]:
pre_tokenizer = pre_tokenizers.WhitespaceSplit()

pre_tokenizer.pre_tokenize_str("We're checking the pre-tokenization step.")

[("We're", (0, 5)),
 ('checking', (6, 14)),
 ('the', (15, 18)),
 ('pre-tokenization', (19, 35)),
 ('step.', (36, 41))]

In [13]:
pre_tokenizer = pre_tokenizers.Sequence(
    [pre_tokenizers.WhitespaceSplit(), pre_tokenizers.Punctuation()]
)

pre_tokenizer.pre_tokenize_str("We're checking the pre-tokenization step.")

[('We', (0, 2)),
 ("'", (2, 3)),
 ('re', (3, 5)),
 ('checking', (6, 14)),
 ('the', (15, 18)),
 ('pre', (19, 22)),
 ('-', (22, 23)),
 ('tokenization', (23, 35)),
 ('step', (36, 40)),
 ('.', (40, 41))]

In [14]:
special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]

trainer = trainers.WordPieceTrainer(vocab_size = 20000, special_tokens = special_tokens, min_frequency = 2)

trainer

In [15]:
tokenizer.train_from_iterator(text_samples, trainer = trainer)

In [16]:
encoding = tokenizer.encode("We're checking pre-tokenization step.")

encoding

Encoding(num_tokens=15, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [17]:
print("Encoding_ids :", encoding.ids)

print("Encoding_tokens :", encoding.tokens)

print("Encoding_offsets :", encoding.offsets)

Encoding_ids : [310, 9, 138, 212, 128, 1164, 913, 13, 114, 74, 111, 1263, 1206, 61, 14]
Encoding_tokens : ['we', "'", 're', 'ch', '##ec', '##king', 'pre', '-', 'to', '##k', '##en', '##ization', 'ste', '##p', '.']
Encoding_offsets : [(0, 2), (2, 3), (3, 5), (6, 8), (8, 10), (10, 14), (15, 18), (18, 19), (19, 21), (21, 22), (22, 24), (24, 31), (32, 35), (35, 36), (36, 37)]


In [18]:
cls_token_id = tokenizer.token_to_id("[CLS]")
sep_token_id = tokenizer.token_to_id("[SEP]")

print(cls_token_id, sep_token_id)

2 3


In [19]:
tokenizer.post_processor = processors.TemplateProcessing(
    single = f"[CLS]:0 $A:0 [SEP]:0",
    pair = f"[CLS]:0 $A:0 [SEP]:0 $B:1 [SEP]:1",
    special_tokens = [("[CLS]", cls_token_id), ("[SEP]", sep_token_id)],
)

In [20]:
encoding = tokenizer.encode("We're checking pre-tokenization step.")

print("Encoding_ids :", encoding.ids)
print()
print("Encoding_tokens :", encoding.tokens)
print()
print("Encoding_offsets :", encoding.offsets)
print()
print("Encoding_type_ids :", encoding.type_ids)

Encoding_ids : [2, 310, 9, 138, 212, 128, 1164, 913, 13, 114, 74, 111, 1263, 1206, 61, 14, 3]

Encoding_tokens : ['[CLS]', 'we', "'", 're', 'ch', '##ec', '##king', 'pre', '-', 'to', '##k', '##en', '##ization', 'ste', '##p', '.', '[SEP]']

Encoding_offsets : [(0, 0), (0, 2), (2, 3), (3, 5), (6, 8), (8, 10), (10, 14), (15, 18), (18, 19), (19, 21), (21, 22), (22, 24), (24, 31), (32, 35), (35, 36), (36, 37), (0, 0)]

Encoding_type_ids : [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [21]:
encoding = tokenizer.encode("We're checking pre-tokenization step.", "on a pair of sentences.")

print("Encoding_ids :", encoding.ids)
print()
print("Encoding_tokens :", encoding.tokens)
print()
print("Encoding_offsets :", encoding.offsets)
print()
print("Encoding_type_ids :", encoding.type_ids)

Encoding_ids : [2, 310, 9, 138, 212, 128, 1164, 913, 13, 114, 74, 111, 1263, 1206, 61, 14, 3, 147, 30, 752, 158, 121, 427, 67, 854, 14, 3]

Encoding_tokens : ['[CLS]', 'we', "'", 're', 'ch', '##ec', '##king', 'pre', '-', 'to', '##k', '##en', '##ization', 'ste', '##p', '.', '[SEP]', 'on', 'a', 'pa', '##ir', 'of', 'sen', '##t', '##ences', '.', '[SEP]']

Encoding_offsets : [(0, 0), (0, 2), (2, 3), (3, 5), (6, 8), (8, 10), (10, 14), (15, 18), (18, 19), (19, 21), (21, 22), (22, 24), (24, 31), (32, 35), (35, 36), (36, 37), (0, 0), (0, 2), (3, 4), (5, 7), (7, 9), (10, 12), (13, 16), (16, 17), (17, 22), (22, 23), (0, 0)]

Encoding_type_ids : [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [22]:
tokenizer.decoder = decoders.WordPiece(prefix = "##")

tokenizer.decode(encoding.ids)

"we ' re checking pre - tokenization step. on a pair of sentences."

In [23]:
tokenizer.save("word_piece_tokenizer.json")

In [24]:
new_tokenizer = Tokenizer.from_file("word_piece_tokenizer.json")

new_tokenizer

In [25]:
encoding = new_tokenizer.encode("We're checking pre-tokenization step.", "on a pair of sentences.")

print("Encoding_ids :", encoding.ids)
print()
print("Encoding_tokens :", encoding.tokens)
print()
print("Encoding_offsets :", encoding.offsets)
print()
print("Encoding_type_ids :", encoding.type_ids)

Encoding_ids : [2, 310, 9, 138, 212, 128, 1164, 913, 13, 114, 74, 111, 1263, 1206, 61, 14, 3, 147, 30, 752, 158, 121, 427, 67, 854, 14, 3]

Encoding_tokens : ['[CLS]', 'we', "'", 're', 'ch', '##ec', '##king', 'pre', '-', 'to', '##k', '##en', '##ization', 'ste', '##p', '.', '[SEP]', 'on', 'a', 'pa', '##ir', 'of', 'sen', '##t', '##ences', '.', '[SEP]']

Encoding_offsets : [(0, 0), (0, 2), (2, 3), (3, 5), (6, 8), (8, 10), (10, 14), (15, 18), (18, 19), (19, 21), (21, 22), (22, 24), (24, 31), (32, 35), (35, 36), (36, 37), (0, 0), (0, 2), (3, 4), (5, 7), (7, 9), (10, 12), (13, 16), (16, 17), (17, 22), (22, 23), (0, 0)]

Encoding_type_ids : [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [26]:
from transformers import PreTrainedTokenizerFast

wrapped_tokenizer = PreTrainedTokenizerFast(
    tokenizer_object = tokenizer,
    unk_token = "[UNK]",
    pad_token = "[PAD]",
    cls_token = "[CLS]",
    sep_token = "[SEP]",
    mask_token = "[MASK]",
)

wrapped_tokenizer.tokenize("We're checking pre-tokenization step.", "on a pair of sentences.")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


['we',
 "'",
 're',
 'ch',
 '##ec',
 '##king',
 'pre',
 '-',
 'to',
 '##k',
 '##en',
 '##ization',
 'ste',
 '##p',
 '.',
 'on',
 'a',
 'pa',
 '##ir',
 'of',
 'sen',
 '##t',
 '##ences',
 '.']

In [27]:
from transformers import BertTokenizerFast

wrapped_tokenizer = BertTokenizerFast(tokenizer_object=tokenizer)

wrapped_tokenizer.tokenize("We're checking pre-tokenization step.", "on a pair of sentences.")

['we',
 "'",
 're',
 'ch',
 '##ec',
 '##king',
 'pre',
 '-',
 'to',
 '##k',
 '##en',
 '##ization',
 'ste',
 '##p',
 '.',
 'on',
 'a',
 'pa',
 '##ir',
 'of',
 'sen',
 '##t',
 '##ences',
 '.']